In [2]:
from rdkit import rdBase, Chem, DataStructs
from rdkit.Chem import AllChem, Draw, PandasTools
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("VDss_dataset_uen.csv", index_col=False)
df.head()

,SMILES,MW,logP,VDss(L/kg),logVDss
0,CC(C)N(CCC(c1ccccc1)c1cc(CO)ccc1O)C(C)C,341.495,4.52530,2.41,0.382017
1,O=C(OC1C[N+]2(CCCOc3ccccc3)CCC1CC2)C(O)(c1cccs...,484.663,4.66680,4.30,0.633468
2,CC[C@H]1C2C[C@H]3[C@@H]4N(C)c5ccccc5[C@]45C[C@...,326.440,1.55450,4.00,0.602060
3,C=CC[N+]12CCC34c5ccccc5N5/C=C6/C7CC8C9(CC[N+]8...,666.910,5.47560,0.32,-0.494850
4,CCCS(=O)CCCN(CC)CC(O)COc1ccc(C#N)cc1,352.500,2.16868,1.80,0.255273


In [4]:
PandasTools.AddMoleculeColumnToFrame(frame=df, smilesCol="SMILES")

In [5]:
# def cal_fingerprint(df, type):
#     if type == "morgan fingerprint":

In [6]:
test_mol1 = df.ROMol[0]
test_mol2 = df.ROMol[1]
fp1 = AllChem.GetMorganFingerprintAsBitVect(test_mol1, 2, 1024)
fp2 = AllChem.GetMorganFingerprintAsBitVect(test_mol2, 2, 1024)
DataStructs.TanimotoSimilarity(fp1, fp2)

0.11392405063291139

In [13]:
df.BitVec[[1, 2]]

1    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: BitVec, dtype: object

In [21]:
morgan_fps = []
df["BitVec"] = df.ROMol.apply(lambda mol: AllChem.GetMorganFingerprintAsBitVect(mol, 2, 1024))

display(DataStructs.TanimotoSimilarity(df.BitVec[0], df.BitVec[1]))
display(DataStructs.DiceSimilarity(df.BitVec[0], df.BitVec[1]))
display(DataStructs.BulkTanimotoSimilarity(df.BitVec[0], list(df.BitVec)[1:6])) # 1st var: the ref molecule, 2nd var: mol list
display(DataStructs.BulkDiceSimilarity(df.BitVec[0], list(df.BitVec)[1:6])) # 1st var: the ref molecule, 2nd var: mol list


0.11392405063291139

0.20454545454545456

[0.11392405063291139,
 0.10526315789473684,
 0.0963855421686747,
 0.136986301369863,
 0.17857142857142858]

[0.20454545454545456,
 0.19047619047619047,
 0.17582417582417584,
 0.24096385542168675,
 0.30303030303030304]